## Lecture2: Linear prediction

### 目标

1. 理解监督式学习
2. 理解线性回归 linear regression（也就是 `最小二乘法`）
3. 理解如何使用线性回归进行预测
4. 学习如何推导最小二乘估计

### 线性监督式学习

1. 许多真实过程可以通过线性模型拟合
2. 线性回归通常作为大型系统的模块出现
3. 线性问题可以通过分析法解决
4. 线性预测涉及到很多机器学习中的核心概念

包含 $n$ 个训练样本的（输入-输出对）数据集合：$\{x_{1:n}, y_{1:n}\}$，其中每一输入数据 $x \in R^{1 \times d}$，拥有 $d$ 个属性的向量，eg：

$$x_{1:n} = \{x_1, x_2, ..., x_n\}$$
$$x_i = [x_{11}, x_{12}, ..., x_{1d}]$$

![](imgs/L2-1.png)

根据训练样本集合$\{x_{1:n}, y_{1:n}\}$我们可以得到一个模型，用于预测新的输入所对应的输出，这个过程就是**训练**或**学习**；对于新的输入值$x_{n+1}$，我们可以得到对应的预测值$ \widehat{y}(x_{n+1}) $

假设输入与输入之间关系的模型为：$\widehat{y}(x_i) = \theta_1 + x_i\theta_2$，我们可以通过**最小化目标函数**的方法来确定模型中的参数值：

$$J(\theta) = \sum_{i=1}^n(y_i - \widehat{y_i})^2 = \sum_{i=1}^n(y_i - \theta_1 - x_i\theta_2)^2$$

### 线性预测

输入与输出之间呈线性关系：

$$\widehat{y_i} = \sum_{j=1}^dx_{ij}\theta_j$$

$= x_{i1}\theta_1 + x_{i2}\theta_2 + ... + x_{id}\theta_d$

用矩阵表示法表示：

$$\mathbf{\widehat{y}} = \mathbf{X\theta} $$

其中：$\widehat{y} \in R^{n \times d}$，$\mathbf{X} \in R^{d \times 1}$，$\mathbf{\theta} \in R^{d \times 1}$

![](imgs/L2-2.png)

### 优化策略

最小化：

$$J(\mathbf{\theta}) = (\mathbf{y-X\theta})^T(\mathbf{y-X\theta}) = \sum_{i=1}^n(y_i - x_i^T\mathbf{\theta})$$

![](imgs/L2-3.png)

### 通过求导求解（最小二乘法）

矩阵求导规则：

$$\frac{\partial \mathbf{A}\theta}{\partial \mathbf{\theta}} = \mathbf{A^T}$$

$$\frac{\partial \mathbf{\theta^TA\theta}}{\partial \mathbf{\theta}} = 2\mathbf{A^T\theta}$$

![](imgs/L2-4.png)

上面给出最小二乘法直接求解的方法，但是$\mathbf{\theta} = \mathbf{(X^TX)^{-1}X^Ty}$，对矩阵求逆是一个相当费时的运算，因此一般采用迭代的方案：**梯度下降法（见附注）**。

### Torch 求解 - 线性回归的例子

**[Demo](https://github.com/torch/demos/blob/master/linear-regression/example-linear-regression.lua)**，笔记见[Blog]()。

In [1]:
-- 0. 首先加载必须的包
require 'torch'
require 'optim'
require 'nn'

In [2]:
-- 1. 训练数据
---- 数据存储在 Torch Tensor (2维数组)，每一行（row）为一条训练样本，每一列（column）为变量（或属性）。
---- 第一列为目标（输出），剩余为输入。

data = torch.Tensor{
   {40,  6,  4},
   {44, 10,  4},
   {46, 12,  5},
   {48, 14,  7},
   {52, 16,  9},
   {58, 18, 12},
   {60, 22, 14},
   {68, 24, 20},
   {74, 26, 21},
   {80, 32, 24}
}

print(#data)

 10
  3
[torch.LongStorage of size 2]



In [10]:
-- 2. 定义模型
---- 这一模型（线性回归）只有一层（one layer），称为 module，两个输入属性，一个输出。
---- 共需要3个参数：y = a0 + a1x1 + a2x2
---- 线性模型必须储存在容器中，在多层模型中，可以采用序列容器（sequential container），
---- 因为前一层的输出将会变成后面一层的输入。
---- 本例中只有一层。

model = nn.Sequential()                 -- 定义容器
ninputs = 2; noutputs = 1
model:add(nn.Linear(ninputs, noutputs)) -- 向容器中添加一个组块（层），本例中只有一个组块。

-- print(help(model.forward))
print(help(model.getParameters))

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++	
[flatParameters, flatGradParameters] getParameters()


This function returns two tensors. One for the flattened learnable
parameters flatParameters and another for the gradients of the energy
wrt to the learnable parameters flatGradParameters .

Custom modules should not override this function. They should instead 
override parameters(...) which is, in turn, called by the present function.

This function will go over all the weights and gradWeights and make them 
view into a single tensor (one for weights and one for gradWeights). Since 
the storage of every weight and gradWeight is changed, this function should 
be called only once on a given network.	
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++	



In [11]:
-- 3. 定义损失函数
---- 最小化目标函数来确定各个参数，
---- 本例中求得模型预测与实际值之间的`均方差`：Mean Square Error（MSE, 见：附注-2）的最小值

-- tips: 可以用 help 打印函数文档，cool~

criterion = nn.MSECriterion()


print(help(criterion.forward))
print(help(criterion.backward))
-- print(help(optim.sgd))

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++	
[output] forward(input, target)


Given an input and a target , compute the loss function associated 
to the criterion and return the result.
In general input and target are  Tensor s, but some specific criterions 
might require some other type of object.

The output returned should be a scalar in general.

The state variable  self.output  should be updated after a call to 
forward() .	
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++	

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++	
[gradInput] backward(input, target)


Given an input and a target , compute the gradients of the loss function 
associated to the criterion and return the result.
In general input , target and gradInput are  Tensor s, but some 
specific criterions might require some other type of object.

The state variable  self.gradInput  should be updated after a call 
to backward() .	
+++++++++++++++++++++

In [7]:
-- 4. 训练模型
---- 通过随机梯度下降法（stochatic gradient descent procedure, SGD）求最小化损失函数的参数
x, dl_dx = model:getParameters()

-- print(x)
-- print(dl_dx)

feval = function(x_new)
    if x ~= x_new then
        x:copy(x_new)
    end
    _nidx_ = (_nidx_ or 0) + 1
    if _nidx_ > (#data)[1] then _nidx_ = 1 end
    
    local sample = data[_nidx_]
    local target = sample[{{1}}]
    local inputs = sample[{{2, 3}}]
    
    dl_dx:zero()
    local loss_x = criterion:forward(model:forward(inputs), target)
    model:backward(inputs, criterion:backward(model.output, target))
    
    return loss_x, dl_dx
end

sgd_params = {
    learningRate = 1e-3,
    learningRateDecay = 1e-4,
    weightDecay = 0,
    momentum = 0
}

for i = 1, 1e4 do
    current_loss = 0
    for i = 1, (#data)[1] do
        
        -- optim.sgd@https://github.com/torch/optim/blob/master/sgd.lua
        _, fs = optim.sgd(feval, x, sgd_params)
        
        current_loss = current_loss + fs[1]
    end
    current_loss = current_loss / (#data)[1]
end
print('current loss = ' .. current_loss)

current loss = 1.5817641763371	


In [8]:
-- 5. 参数结果
print(model:getParameters())

  0.6667
  1.1154
 31.6374
[torch.DoubleTensor of size 3]

-23.2280
-17.4210
 -0.7259
[torch.DoubleTensor of size 3]




In [9]:
-- 6. 预测 & 测试
test = {40.32, 42.92, 45.33, 48.85, 52.37, 57, 61.82, 69.78, 72.19, 79.42}
print('id\tapprox\ttext')
for i = 1, (#data)[1] do
    local myPrediction = model:forward(data[i][{{2,3}}])
    print(string.format("%2d\t%.2f\t%.2f", i, myPrediction[1], test[i]))
end

id	approx	text	
 1	40.10	40.32	
 2	42.77	42.92	
 3	45.22	45.33	
 4	48.78	48.85	
 5	52.34	52.37	
 6	57.02	57.00	
 7	61.92	61.82	
 8	69.95	69.78	
 9	72.40	72.19	
10	79.74	79.42	


## 附注

### 1. 随机梯度下降

**什么是梯度下降法则？**

通过求导，让参数的变化沿着_误差曲面_最陡峭的方向发展，直到达到最低点。所谓误差曲面即参数假设空间里面目标函数构成的曲面，如：

![](imgs/L2-3.png)

**梯度下降法的推导：**

模型$h(x)$公式为：
$$h(x) = h_\theta(x) = \sum_{i=1}^d\theta_ix_i = \vec{\theta}^T\vec{X}$$
目标函数（损失函数）：
$$J(\vec{\theta}) = \frac{1}{2}\sum_{i=1}^d(h_\theta(x) - y_i)^2$$

所谓梯度下降，是指按照$J(\vec{\theta})$各个维度的导数方向减少，直到目标函数到达最小值（可能是局部最小值）。在迭代过程中，对于每一个系数$\theta_i$：

$$\theta_i = \theta_i - \alpha \frac{\partial}{\partial \theta_i} J(\theta)$$

对于$j \neq i$：

$$\frac{\partial}{\partial \theta_i}J(\theta) = (h_\theta(x) - y)\frac{\partial}{\partial \theta_i} h_\theta(x) = (h_\theta(x) - y)x_i$$

，于是：

$$\theta_i = \theta_i - \alpha (h_\theta(x) - y)x_i$$

，其中$\alpha$为学习率（Learning Rate）。

**梯度下降算法：**

```lua
GD(traning_examples, lr) -- lr is learning rate
    training_examples 是 {inputs, output}
    初始化 w_i 为某个随机值
    终止条件之前，迭代：
        初始化 delta_w_i = 0
        对于 training_samples 中每一条样本数据，do
            将 inputs 输入，计算输出为 o
            对于每个 w_i，do
                delta_w_i = delta_w_i + lr*(output - o)*x_i -- step A
        对于每个 w_i，do
            w_i = w_i + delta_w_i                      -- step B
```
> SGD 算法，去掉 `step B`，并将 `step A` 替换为 `w_i = w_i + lr * (output - o) * x_i`。

#### 2. MSE

均方差（MSE）：https://en.wikipedia.org/wiki/Mean_squared_error
$$MSE = \frac{1}{n} \sum_{i=1}^n(\hat{Y_i} - Y_i)^2$$

#### 3. 向量的导数 & 偏导数

## 参考链接

1. [知乎：最小二乘法和梯度下降法有哪些区别？](https://www.zhihu.com/question/20822481)
2. [机器学习中的数学(1)-回归(regression)、梯度下降(gradient descent)](http://www.cnblogs.com/LeftNotEasy/archive/2010/12/05/mathmatic_in_machine_learning_1_regression_and_gradient_descent.html)